https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning

### Errors
- not masking the pad inputs for backprop
`

# Things you're trying and may need to change back
- batchnorm eval on cnn during training
- tanh non-linearity when projecting cnn output to embedding space
- using cnn output as first input to rnn instead of as hidden state -- what about SOS?
- Train embeddings or don't use w2v?

In [1]:
import torch
import re, pickle, collections, numpy as np, keras, math, operator, pdb
import matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.autograd import Variable
from torch import optim
from torchvision.models import inception_v3
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import re
from os import path
from glob import glob
from torch.utils.data import DataLoader
from PIL import Image
from data_loader import get_loader
from build_vocab import build_vocab
from encoder import Encoder
from torch.nn.utils.rnn import pack_padded_sequence
from utils import *
from create_emb_matrix import create_emb_mat
from decoder import LSTMDecoder
from tqdm import tnrange, tqdm_notebook
import pdb
%load_ext autoreload
%autoreload 2


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN Mixed dnn version. The header is from one version, but we link with a different version (5103, 7003))
/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [2]:
- You have tuples of image names and captions. 
- Pass those into your custom dataset. Along with your transform, vocab and root directory.
- implement the getitem function that loads and transforms the image. And also tokenizes and encodes the caption.
- then write your collate function.
- and get_loader function.

SyntaxError: invalid syntax (<ipython-input-2-988a52345229>, line 1)

In [8]:
dpath = 'flikr8/'
imgpath =  'flikr8/Flicker8k_Dataset/' 
tr_img_path = imgpath +'train/wrap'
dev_img_path = imgpath +'dev/wrap'
#samp_tr_img_path = imgpath + 'samp/train/wrap/'
MODEL_PREFIX = 'lstm_do'
BATCH_SIZE = 32

In [4]:
tr_caps = pickle.load(open(dpath+'train_first_caps.pkl', 'rb'))

In [4]:
samp_caps = make_samp_caps(tr_caps, samp_tr_img_path+'*.jpg'); len(samp_caps)

32

In [5]:
vocab = build_vocab(tr_caps, 0)

[0/6000] Tokenized the captions.
[1000/6000] Tokenized the captions.
[2000/6000] Tokenized the captions.
[3000/6000] Tokenized the captions.
[4000/6000] Tokenized the captions.
[5000/6000] Tokenized the captions.


In [6]:
enc = Encoder(batch_size=BATCH_SIZE, emb_size=300, pool_size=8, fc_in_size=2048)
dec = LSTMDecoder(len(vocab), 300, 2)

if torch.cuda.is_available(): 
    enc.cuda()
    dec.cuda()

LR = 0.001
params = list(dec.parameters()) + list(enc.linear.parameters()) + list(enc.bn.parameters())
optimizer = torch.optim.Adam(params, lr=LR)


In [24]:
crit = nn.CrossEntropyLoss()

for epoch in range(1000):
    dl = get_loader(tr_img_path, tr_caps, vocab, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
    tot_loss = 0.0
    for i, (imgs, caps, ls) in enumerate(dl):
        optimizer.zero_grad()
        #dl.dataset.sanity(imgs, caps, ls)
        #pdb.set_trace()
        imgs = to_var(imgs, volatile=True); caps = to_var(caps) #VOLATILE ON IMGS MAYBE?
        targets = pack_padded_sequence(caps, ls, batch_first=True)[0]
        imgs_enc = enc(imgs)

        out = dec(imgs_enc, caps, ls)
        loss = crit(out, targets)
        loss.backward()
        tot_loss += loss
        optimizer.step()
    torch.save(enc.state_dict(), 'models/enc_{}.pt'.format(MODEL_PREFIX))
    torch.save(dec.state_dict(), 'models/dec_{}.pt'.format(MODEL_PREFIX))
    torch.save(optimizer.state_dict(), 'models/opt_{}.pt'.format(MODEL_PREFIX))
    print('epoch {} avg. loss: {}'.format(epoch, tot_loss.data[0] / i))
    

epoch 0 avg. loss: 4.497527419879872
epoch 1 avg. loss: 3.5567334903183805
epoch 2 avg. loss: 3.2433970461609545
epoch 3 avg. loss: 3.0364156743531585
epoch 4 avg. loss: 2.8720499674479165
epoch 5 avg. loss: 2.7310599050214215
epoch 6 avg. loss: 2.6007933257728495
epoch 7 avg. loss: 2.4739506219023015
epoch 8 avg. loss: 2.353401102045531
epoch 9 avg. loss: 2.231714392221102
epoch 10 avg. loss: 2.1142674928070395
epoch 11 avg. loss: 1.994261710874496
epoch 12 avg. loss: 1.878323954920615
epoch 13 avg. loss: 1.7615663261823757
epoch 14 avg. loss: 1.6453179800382225
epoch 15 avg. loss: 1.531279533140121
epoch 16 avg. loss: 1.4222271006594422
epoch 17 avg. loss: 1.3191927838069137
epoch 18 avg. loss: 1.2210003432407175
epoch 19 avg. loss: 1.1204652683709257
epoch 20 avg. loss: 1.032291289298765
epoch 21 avg. loss: 0.9482222526304184
epoch 22 avg. loss: 0.8675655241935484
epoch 23 avg. loss: 0.7928936866021925
epoch 24 avg. loss: 0.7237726847330729
epoch 25 avg. loss: 0.6606338665049564
epo

Process Process-68:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/nbs/data_loader.py", line 47, in __getitem__
    image = Image.open(os.path.join(self.root, path)).convert('RGB')
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/PIL/Image.py", line 860, in convert
    self.load()
  File "/home/ubuntu/anaconda2/envs/py36/lib

KeyboardInterrupt: 

### Train Sampling

In [ ]:
dl = get_loader(tr_img_path, tr_caps, vocab, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
imgs, caps = sampleBatch(enc, dec, dl)
displaySamples(imgs, caps, vocab)

### Dev sampling

In [13]:
dev_caps = pickle.load(open(dpath+'dev_first_caps.pkl', 'rb'))

In [ ]:
dl = get_loader(dev_img_path, dev_caps, vocab, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
imgs, caps = sampleBatch(enc, dec, dl)
displaySamples(imgs, caps, vocab)

### Dev Loss

In [7]:
enc.load_state_dict(torch.load('models/enc_{}.pt'.format(MODEL_PREFIX)))
dec.load_state_dict(torch.load('models/dec_{}.pt'.format(MODEL_PREFIX)))

In [18]:
def validate_dev(enc, dec, dev_caps=dev_caps, dev_img_path=dev_img_path, batch_size=BATCH_SIZE):
    enc.eval(); dec.eval()
    crit = nn.CrossEntropyLoss()
    dl = get_loader(dev_img_path, dev_caps, vocab, batch_size=batch_size, shuffle=True, num_workers=1)
    
    tot_loss = 0.0
    for i, (imgs, caps, ls) in enumerate(dl):
        #dl.dataset.sanity(imgs, caps, ls)
        #pdb.set_trace()
        imgs = to_var(imgs, volatile=True); caps = to_var(caps) #VOLATILE ON IMGS MAYBE?
        targets = pack_padded_sequence(caps, ls, batch_first=True)[0]
        imgs_enc = enc(imgs)

        out = dec(imgs_enc, caps, ls)
        loss = crit(out, targets)
        tot_loss += loss
    enc.train(); dec.train()
    return tot_loss.data[0] / i

In [19]:
enc = Encoder(batch_size=BATCH_SIZE, emb_size=300, pool_size=8, fc_in_size=2048)
dec = LSTMDecoder(len(vocab), 300, 2)

if torch.cuda.is_available(): 
    enc.cuda()
    dec.cuda()

LR = 0.001
params = list(dec.parameters()) + list(enc.linear.parameters()) + list(enc.bn.parameters())
optimizer = torch.optim.Adam(params, lr=LR)


In [20]:
crit = nn.CrossEntropyLoss()

for epoch in range(1000):
    dl = get_loader(tr_img_path, tr_caps, vocab, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
    best_loss = 1000.0
    tot_loss = 0.0
    for i, (imgs, caps, ls) in enumerate(dl):
        optimizer.zero_grad()
        
        #dl.dataset.sanity(imgs, caps, ls)
        #pdb.set_trace()
        imgs = to_var(imgs, volatile=True); caps = to_var(caps) #VOLATILE ON IMGS MAYBE?
        targets = pack_padded_sequence(caps, ls, batch_first=True)[0]
        imgs_enc = enc(imgs)

        out = dec(imgs_enc, caps, ls)
        loss = crit(out, targets)
        loss.backward()
        tot_loss += loss
        optimizer.step()
    avg_tr_loss = tot_loss.data[0] / (i +1)
    print('epoch {} avg. loss: {}'.format(epoch, avg_tr_loss))
    if (epoch + 1) % 5 == 0:
        avg_val_loss =  validate_dev(enc, dec)
        print('epoch {} avg val loss: {}'.format(epoch, avg_val_loss))
        if avg_val_loss < best_loss:
            torch.save(enc.state_dict(), 'models/enc_{}.pt'.format(MODEL_PREFIX))
            torch.save(dec.state_dict(), 'models/dec_{}.pt'.format(MODEL_PREFIX))
            torch.save(optimizer.state_dict(), 'models/opt_{}.pt'.format(MODEL_PREFIX))
            best_loss = avg_val_loss
        

epoch 0 avg. loss: 4.732233220880682
epoch 1 avg. loss: 3.652359743169285
epoch 2 avg. loss: 3.366281764392547
epoch 3 avg. loss: 3.1827960498830214
epoch 4 avg. loss: 3.038723379532921
epoch 4 avg val loss: 3.345093282063802
epoch 5 avg. loss: 2.932257177995488
epoch 6 avg. loss: 2.8398555000835564
epoch 7 avg. loss: 2.759361552682152
epoch 8 avg. loss: 2.6880901153074865
epoch 9 avg. loss: 2.6238199325806315
epoch 9 avg val loss: 3.2283045450846353
epoch 10 avg. loss: 2.5684368929123496
epoch 11 avg. loss: 2.511904629794034
epoch 12 avg. loss: 2.4577468627276904
epoch 13 avg. loss: 2.412554572610294
epoch 14 avg. loss: 2.364499341995321
epoch 14 avg val loss: 3.230982716878255
epoch 15 avg. loss: 2.318453538864054


Process Process-37:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/nbs/data_loader.py", line 49, in __getitem__
    image = self.transform(image)
  File "/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py", line 42, in __call__
    img = t(img)
  File "/home/ubuntu/anaco

KeyboardInterrupt: 